## Career-to-degree matcher
This notebook is intended to figure out the best way to retrieve suitable careers offered by ASU online when you start with a list of potential careers.

### Setting up elastic search
We run elasticsearch locally via docker:


```bash
docker-compose up -d elasticsearch
```

Make sure you run this before running this notebook.



Also make sure you do:

```bash
pip uninstall elasticsearch
pip install elasticsearch==8.13.0
```

you are running on Docker ElasticSearch version 8.17.6 and for that one you need to install a compatible 8x library.


### Other python libraries
Also make sure you add the sentence transformer library via:

```bash
pip install fastembed
```

In [1]:
import json
from tqdm.auto import tqdm
from fastembed import TextEmbedding
from elasticsearch import Elasticsearch

/Users/anicol11/ASU Dropbox/Andrea Nicolas Flores/Mac/Documents/projects/alpaca-api/dev/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the degree data from that json file
degree_data_filename = "../datasets/degree_data_all_wCareers.json"
with open(degree_data_filename, "r") as f:
        documents = json.load(f)

In [3]:
# set up embedding model
model_name = 'jinaai/jina-embeddings-v2-small-en'
model  = TextEmbedding(model_name=model_name)

In [18]:
# embed the data inside the documents
operations = []
for doc in tqdm(documents, desc="Embedding degrees"):
    # create two strings, one with degree description and another one with the list of careers
    title_description = doc['degreeTitle'] + " " + doc['shortDescription'] + " " + doc['longDescription']
    careers = ", ".join(doc['careers'])

    # embed them into vectors
    doc['description_vector'] = list(model.embed(title_description))[0].tolist()
    doc['career_vector'] = list(model.embed(careers))[0].tolist()

    # append to a new list
    operations.append(doc)

Embedding degrees: 100%|██████████| 373/373 [00:14<00:00, 25.72it/s]


In [7]:
es_client = Elasticsearch(
    "http://localhost:9200",
)

In [9]:
# set up mapping
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "degreeTitle": {"type": "text"},
            "shortDescription": {"type": "text"},
            "longDescription": {"type": "text"},
            "careers": {"type": "keyword"},
            "description_vector": {
                "type": "dense_vector",
                "dims": 512,           # Must match your model (e.g. jina-v2-small)
                "index": True,
                "similarity": "cosine"
            },
            "career_vector": {
                "type": "dense_vector",
                "dims": 512,           # Must match your model (e.g. jina-v2-small)
                "index": True,
                "similarity": "cosine"  # or "l2_norm"
            }
        }
    }
}


index_name = "degree_information"

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)
else:
    print(f"Index '{index_name}' already exists.")

In [22]:
# populate elastic search with the dadta
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 373/373 [00:01<00:00, 252.30it/s]


### Sample quiz response
Here you will find:
- the responses the user gave to the questions.
- the careers suggested by openAI.

In [ ]:
answers = [
    {
        "id": 1,
        "question": "Where are you in your professional development?",
        "selections": [
            "early-career professional"
        ]
    },
    {
        "id": 2,
        "question": "What are your main interests or passions?",
        "selections": [
            "Technology",
            "Science and Research"
        ]
    },
    {
        "id": 3,
        "question": "What skills do you feel most confident in?",
        "selections": [
            "Technical Skills",
            "Analytical Skills"
        ]
    },
    {
        "id": 4,
        "question": "What type of work environment do you prefer?",
        "selections": [
            "Remote work"
        ]
    },
    {
        "id": 5,
        "question": "What are your career goals or motivations for pursuing a new degree?",
        "selections": [
            "Advancing in my current field"
        ]
    },
    {
        "id": 6,
        "question": "What industries are you most interested in working in?",
        "selections": [
            "Information Technology"
        ]
    },
    {
        "id": 7,
        "question": "What type of job roles are you most interested in pursuing?",
        "selections": [
            "Software Development",
            "Data Analysis"
        ]
    },
    {
        "id": 8,
        "question": "What aspects of a job do you find most rewarding?",
        "selections": [
            "Solving complex problems",
            "Working with cutting-edge technology"
        ]
    },
    {
        "id": 9,
        "question": "What type of side gigs or freelance work are you interested in exploring?",
        "selections": [
            "Freelance coding or development",
            "Consulting"
        ]
    },
    {
        "id": 10,
        "question": "What level of education are you aiming to achieve with your new degree?",
        "selections": [
            "Master's Degree"
        ]
    }
]

recommendations = [
    "Software Engineer",  
    "Data Scientist",
    "IT Consultant",
    "Systems Analyst",
    "Technical Project Manager",
    "DevOps Engineer",
    "Machine Learning Engineer",
    "Web Developer"
]

In [27]:
# set up user profile
user_profile = []
for answer in answers:
    user_profile.append(answer["question"] + " " + ", ".join(answer["selections"]))

user_profile = "; ".join(user_profile)
print(user_profile)

Where are you in your professional development? early-career professional; What are your main interests or passions? Technology, Science and Research; What skills do you feel most confident in? Technical Skills, Analytical Skills; What type of work environment do you prefer? Remote work; What are your career goals or motivations for pursuing a new degree? Advancing in my current field; What industries are you most interested in working in? Information Technology; What type of job roles are you most interested in pursuing? Software Development, Data Analysis; What aspects of a job do you find most rewarding? Solving complex problems, Working with cutting-edge technology; What type of side gigs or freelance work are you interested in exploring? Freelance coding or development, Consulting; What level of education are you aiming to achieve with your new degree? Master's Degree


In [28]:
# select career to search
query = recommendations[1]
print(query)

Data Scientist


In [63]:
def elastic_search(career, profile,model=[]):
    # convert queries to vectors too
    career_vector = list(model.embed(query))[0].tolist()
    profile_vector = list(model.embed(profile))[0].tolist()

    # generate search query
    search_query = {
        "size": 5,
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "should": [
                            {
                                "multi_match": {
                                    "query": career,
                                    "fields": ["careers.text^5", "degreeTitle^2", "longDescription"],
                                    "type": "best_fields",
                                    "fuzziness": "AUTO"
                                }
                            },
                            {
                                "multi_match": {
                                    "query": profile,
                                    "fields": ["longDescription", "shortDescription"],
                                    "type": "best_fields"
                                }
                            }
                        ],
                        "minimum_should_match": 1
                    }
                },
                "script": {
                    "source": """
                        0.25 * cosineSimilarity(params.query_vector_desc, 'description_vector') + 
                        0.75 * cosineSimilarity(params.query_vector_career, 'career_vector') + 1.0
                    """,
                    "params": {
                        "query_vector_desc": profile_vector,
                        "query_vector_career": career_vector
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return response["hits"]["hits"]


In [64]:
search_results = elastic_search(query,user_profile,model=model)

In [73]:
search_results

[{'_index': 'degree_information',
  '_id': 'EgGi9pcBR-58WZl3ZA5B',
  '_score': 1.8358114,
  '_source': {'degreeTitle': 'Online Master of Science in Program Evaluation and Data Analytics',
   'shortDescription': 'Are you passionate about applying data insights to drive organizational performance improvements? This online Master of Science in program evaluation and data analytics from Arizona State University prepares you to make a positive social impact through data and evidence-based decision-making. Upon graduating, your critical thinking and analytical skills will be in high demand by employers across all industries, particularly in the public and nonprofit sectors.',
   'longDescription': 'This program evaluation and data analytics master’s builds interdisciplinary expertise at the intersection of statistics, computer science and management. You’ll learn how to use a variety of data tools and implement rigorous approaches to collecting, analyzing and interpreting data within public 

In [74]:
recommended_degrees_data = []
for result in search_results:
    degree_data = {}
    degree_data["degree_title"] = result['_source']['degreeTitle']
    degree_data["careers"] = result['_source']['careers']
    degree_data["degree_description"] = result['_source']['shortDescription']
    recommended_degrees_data.append(degree_data)


In [76]:
recommended_degrees = json.dumps(recommended_degrees_data,indent=2)
print(recommended_degrees)

[
  {
    "degree_title": "Online Master of Science in Program Evaluation and Data Analytics",
    "careers": [
      "Data ambassador",
      "Data analyst",
      "Data fellow",
      "Data programmer",
      "Data science engineer",
      "Database-driven website consultant",
      "Program performance and evaluation manager",
      "Programmer analyst",
      "Senior analyst in performance assessment"
    ],
    "degree_description": "Are you passionate about applying data insights to drive organizational performance improvements? This online Master of Science in program evaluation and data analytics from Arizona State University prepares you to make a positive social impact through data and evidence-based decision-making. Upon graduating, your critical thinking and analytical skills will be in high demand by employers across all industries, particularly in the public and nonprofit sectors."
  },
  {
    "degree_title": "Online Master of Science in Information Technology (IT)",
   

### Final degree recommendation

In [ ]:
SYSTEM_PROMPT = """ 

You are a career counselor with access to a list called "degrees_data" that contains several degree options. These degrees were selected based on a user's answers to a "career_quiz" and their chosen "career_selection".

Your goal is to recommend the top 3 degrees from "degrees_data" that best match the user's profile and career interest.

Prioritize recommending degree types that:
1. Have a matching "career_selection" inside "careers".
2. Align with the user’s expressed interest — for example, recommend bachelor's degrees if the user is interested in a bachelor's program, master's degrees if they're interested in graduate study (masters or phd), or certificates if they selected a short-term credential.

However, you may recommend a different type of degree (e.g., a master's or certificate instead of a bachelor's) if it is clearly a better fit for the user's career goals and "career_selection", or if there are few relevant options of their preferred type.

Return your answer in the following JSON format — do not include any extra text, explanations, markdown, or code blocks:

[
    {
        "top_choice_1": "<Degree title>",
        "reasoning": "<Concise explanation of why this degree fits the user's selected career>"
    },
    {
        "top_choice_2": "<Degree title>",
        "reasoning": "<Concise explanation of why this degree fits the user's selected career>"
    },
    {
        "top_choice_3": "<Degree title>",
        "reasoning": "<Concise explanation of why this degree fits the user's selected career>"
    }
]


If you think that none of the degrees are a good fit for the user, then return the following:

[
    {
        "top_choice_1": "NONE",
        "reasoning": "<Concise explanation of why you think none of the degrees were a good fit.>"
    },

]

The output must be valid JSON and fully parsable using `json.loads()` in Python.

When wording your reasonings for fit/no fit, talk as if you were talking to the user in a friendly and professional manner.
"""


In [89]:
USER_PROMPT = """ 

<career_selection>
{career_selection}
</career_selection>

<career_quiz>
{career_quiz}
</career_quiz>

<degrees_data>
{degrees_data}
</degrees_data>
"""

In [90]:
selections_user_prompt = USER_PROMPT.format(career_selection=query,career_quiz=user_profile,degrees_data=recommended_degrees)

#### Setting up llm

In [91]:
from openai import OpenAI
from dotenv import load_dotenv

In [92]:
# Load environment variables from .env file
load_dotenv()
# load openai key
client = OpenAI()

In [93]:
# set up fcn to call openai
def llm(user_prompt,system_prompt="you are a helpful assistant",model="gpt-4o-mini",temperature=0.5):
    ''' this function calls the openAI api and feeds it user/system prompts'''
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        )
    return response.choices[0].message.content

In [94]:
#actually call the LLM
response = llm(user_prompt = selections_user_prompt,system_prompt=SYSTEM_PROMPT,model="gpt-4o-mini",temperature=0.5)

In [104]:
print(response)

[
    {
        "top_choice_1": "Online Master of Computer Science – Big Data Systems",
        "reasoning": "This degree directly aligns with your interest in data science and equips you with the skills to design systems that handle large datasets, which is essential for a career as a Data Scientist."
    },
    {
        "top_choice_2": "Online Master of Science in Information Technology (IT)",
        "reasoning": "This program offers a strong foundation in data science and technology leadership, preparing you for roles that require both analytical and technical skills, making it a great fit for your career goals."
    },
    {
        "top_choice_3": "Online Master of Science in Program Evaluation and Data Analytics",
        "reasoning": "This degree focuses on applying data insights for performance improvements, which complements your analytical skills and desire to solve complex problems in the field of data science."
    }
]


In [96]:
# extract bot recommendations
recommendations_bot = json.loads(response)
print(recommendations_bot)

[{'top_choice_1': 'Online Master of Computer Science – Big Data Systems', 'reasoning': 'This degree directly aligns with your interest in data science and equips you with the skills to design systems that handle large datasets, which is essential for a career as a Data Scientist.'}, {'top_choice_2': 'Online Master of Science in Information Technology (IT)', 'reasoning': 'This program offers a strong foundation in data science and technology leadership, preparing you for roles that require both analytical and technical skills, making it a great fit for your career goals.'}, {'top_choice_3': 'Online Master of Science in Program Evaluation and Data Analytics', 'reasoning': 'This degree focuses on applying data insights for performance improvements, which complements your analytical skills and desire to solve complex problems in the field of data science.'}]


In [103]:
i = 3
print(recommendations_bot[i-1][f'top_choice_{i}'])
print(recommendations_bot[i-1][f'reasoning'])

Online Master of Science in Program Evaluation and Data Analytics
This degree focuses on applying data insights for performance improvements, which complements your analytical skills and desire to solve complex problems in the field of data science.
